In [20]:

pip install tqdm



[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.0 MB/s eta 0:00:000:00:010:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 8.0 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [42]:
pip install janome


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 7.6 MB/s eta 0:00:00m eta 0:00:010:01:01

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [50]:
from janome.tokenizer import Tokenizer as JanomeTokenizer
import fitz  # PyMuPDF
from tqdm.auto import tqdm
import re

# Tokenizer for Japanese text
janome_tokenizer = JanomeTokenizer()


# Define a function to further preprocess Japanese text
def preprocess_text_japanese_advanced(text):
    # Handle special characters and normalization
    cleaned_text = text.replace(
        "\xa0", " "
    ).strip()  # Replace special characters as needed
    cleaned_text = re.sub(r"\s+", " ", cleaned_text)  # Normalize whitespace

    # Tokenization with Janome
    tokens = [token.surface for token in janome_tokenizer.tokenize(cleaned_text)]

    # Additional steps (example: handling numerals)
    # Example: Replace numerals with a placeholder
    tokens = [re.sub(r"\d+", "<NUM>", token) for token in tokens]

    # Join tokens back into cleaned text
    cleaned_text = " ".join(tokens)

    return cleaned_text


# Function to read PDF and preprocess text
def open_and_read_pdf_japanese_advanced(
    pdf_path: str, output_file: str = "data_preprocessed.txt"
) -> list[dict]:
    doc = fitz.open(pdf_path)
    pages_and_texts = []
    with open(output_file, "w", encoding="utf-8") as f:
        for page_number, page in tqdm(
            enumerate(doc), total=len(doc), desc="Reading PDF"
        ):
            text = page.get_text()
            cleaned_text = preprocess_text_japanese_advanced(
                text
            )  # Preprocess the extracted text
            pages_and_texts.append(
                {
                    "page_number": page_number,
                    "page_char_count": len(cleaned_text),
                    "page_word_count": len(cleaned_text.split()),
                    "page_sentence_count_raw": len(
                        cleaned_text.split("。")
                    ),  # Japanese period
                    "page_token_count": len(
                        cleaned_text.split()
                    ),  # Assuming tokens as words
                    "text": cleaned_text,
                }
            )
            # Write cleaned text to file
            f.write(f"Page {page_number}:\n")
            f.write(cleaned_text + "\n\n")

    return pages_and_texts


# Example usage
pdf_path = "data.pdf"
output_file = "data_preprocessed.txt"
pages_and_texts = open_and_read_pdf_japanese_advanced(pdf_path, output_file)

Reading PDF: 100%|██████████| 23/23 [00:00<00:00, 23.46it/s]


In [51]:
import random

random.sample(pages_and_texts, k=1)

[{'page_number': 16,
  'page_char_count': 1172,
  'page_word_count': 374,
  'page_sentence_count_raw': 9,
  'page_token_count': 374,
  'text': '<NUM>   【 卒業 時 ・ 卒業 後 の 検証 ・ 評価 について 】   本学 卒業 時 に ディプロマポリシー （ 卒業 認定 ・ 学位 授与 の 方針 ） を 満たす 人材 と なっ た か について 、 次   の <NUM>   項目 について 検証 ・ 評価 し て い ます 。 検証 用 データ として は 、 学内 で の 調査 及び 大学 IR   調査 の 卒業   生 調査 結果 を 使用 し て い ます 。   ①   修業 年限 で の 卒業 率   修業 年限 は 、 大学 は <NUM>   年 （ <NUM>   年次 編入 学生 は <NUM>   年 ）   、 短期大学 部 は <NUM>   年 （ 長期 履修 学生 は <NUM>   年 又は <NUM>   年 ） で 、 在学 期間 は 修業 年限 の <NUM> 倍 まで と さ れ て い ます 。   大学 に <NUM>   年 以上 在学 し 、   卒業 要件 の 単位 を 優秀 な 成績 で 修得 し た 場合 に は 、   <NUM>   年 未満 の 在学 で の 卒業   （ 早期 卒業 ） が 認め られる 場合 が あり ます 。   ・ <NUM>   年度 卒業生 における 、 学部 別 の 、 修業 年限 内 で の 卒業 者 数 、 卒業 率 （ ※ ） は 次 の とおり です 。   （ ※ 卒業 率 ＝ 修業 年限 内 で の 卒業生 数 ／ 修業 年限 前 の 入 学生 数 ）   学部   卒業 者 数 （ 人 ）   卒業 率   文学部   <NUM>   <NUM> . <NUM> %   経営 情報 学部   <NUM>   生活 科学 部   <NUM>   看護 学部   <NUM>   短期大学 部   <NUM>   <NUM> . <NUM> %   ＜ 検証 ・ 評価 結果 ＞   修業 年限 内 で の 卒業 率 

In [52]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,0,1080,340,6,340,<NUM> 四国大学 アセスメント ポリシー による 情報 の 公表 令和 <NUM...
1,1,1437,503,16,503,<NUM> 【 入学 前 ・ 入学 時 の 検証 ・ 評価 について 】 四国 大学...
2,2,1076,341,7,341,<NUM> ② 志願 理由 総合 型 選抜 入試 における 受験生 の 志願 理...
3,3,1737,535,14,535,<NUM> 【 在学 中 の 学修 状況 の 検証 ・ 評価 について 】 四国 大...
4,4,767,234,5,234,<NUM> ・ <NUM> 年度 の 学部 ・ 学年 別 、 年間 平均 修得 ...


In [53]:
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,23.00,23.00,23.00,23.00,23.00
mean,11.00,1358.43,403.22,6.91,403.22
std,6.78,433.08,120.02,3.72,120.02
min,0.00,395.00,123.00,2.00,123.00
25%,5.50,1126.00,340.50,4.50,340.50
50%,11.00,1437.00,422.00,7.00,422.00
75%,16.50,1710.50,507.00,9.00,507.00
max,22.00,2199.00,573.00,16.00,573.00


In [58]:
pip install -U pip setuptools wheel && pip install -U spacy && python -m spacy download en_core_web_sm && python -m spacy download ja_core_news_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.3 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.4/863.4 kB 6.4 MB/s eta 0:00:006.4 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.41.3
    Uninstalling wheel-0.41.3:
      Successfully uninstalled wheel-0.41.3
  Attempting uninstall: setuptools
    Found existing installation: setuptools 68.2.2
    Uninstalling setuptools-68.2.2:
      Successfully uninstalled setuptools-68.2.2
  Attempting uninstall: pip
    Found existing installation: pip 23.3.1
    Uninstalling pip-23.3.1:
      Successfully uninstalled pip-23.3.1
/Users/yabindrabhujel/shikoku-mobile-app/chatbot/datacollection/venv/bin/python: No module named spacy
Note: you may need to restart the kernel to use updated packages.


In [59]:
import spacy
from spacy.lang.ja import Japanese

# Load the small Japanese language model
nlp = spacy.load("ja_core_news_sm")

# Add the sentencizer to the pipeline
nlp.add_pipe("sentencizer")

# Sample Japanese text for testing
text = "これは文です。これは別の文です。"

doc = nlp(text)
list(doc.sents)

OSError: [E050] Can't find model 'ja_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.